# HW 1

>Or Livne ID 203972922

>Roy Rubin ID 201312907

## Task 1: CSV and SQL
1. Write Python code to create SQLite database “mydb.db” and create a table “mydata” with the schema of the “mydata.csv”
2. Write Python code to load “mydata.csv” into “mydata” table.
3. Write 2 different SQL statements with different condiJons to retrieve different rows. Explain which parts of the statement are predicate and which parts are projecJon.

## imports

In [1]:
import sqlite3
import pandas as pd
import numpy as np 
import random
import os 
import gc
import pyarrow.parquet as pq
import pyarrow as pa
import dask
from dask.dataframe import to_parquet
import csv
from dask.dataframe import from_pandas
import gc 
gc.collect()

0

## define function

In [4]:
"""
Create local CSV file “mydata.csv” with 1000000 
rows with columns (id, fruit, price, color). 
fruit has one of the values ['Orange', 'Grape', 'Apple', 'Banana','Pineapple', 'Avocado'] 
colors are ['Red', 'Green', 'Yellow', 'Blue']. 
Price should be random integer between 10 and 100. 
Filed id should be an index number starting from 1.
"""
class single_fruit():
    fruit_name = ''
    fruit_price = ''
    fruit_color = ''
    fruit_id = ''
    
    # Set the value options
    fruit_options_list = ['Orange', 'Grape', 'Apple', 'Banana','Pineapple', 'Avocado']
    colors_options_list = ['Red', 'Green', 'Yellow', 'Blue']
    price_options_range = [10,100]
    
    def __init__(self, id):
        self.fruit_name = random.choice(self.fruit_options_list)	
        self.fruit_price = np.random.randint(self.price_options_range[0], self.price_options_range[1])
        self.fruit_color = random.choice(self.colors_options_list)
        self.fruit_id = id
        

def init_data_set_configuration():
    # Set the value options
    global max_rows, csv_columns, db_file_name, csv_file_name, columns_type
    global parquet_file_name_using_dask,parquet_file_name_using_pyarray 
    global parquet_file_name_using_pandas
    max_rows = 10
    csv_columns = ['id', 'fruit', 'price', 'color']
    columns_type = ['integer', 'text', 'integer', 'text']

    db_file_name = 'mydata.db'
    csv_file_name = 'mydata.csv'
    parquet_file_name_using_dask = 'mydatapyarrow_dask.parquet'
    parquet_file_name_using_pyarray = 'mydatapyarrow_pyarray.parquet'
    parquet_file_name_using_pandas = 'mydatapyarrow_pandas.parquet'

    return

def create_csvdatabase_file(max_rows):
    # this loop generate single fruit
    create_n_list_in_advance = max_rows*[None]
    for i_row_index in range(max_rows):
        i_friut = single_fruit(i_row_index)
        create_n_list_in_advance[i_row_index] = [i_friut.fruit_id, i_friut.fruit_name, 
                                                 i_friut.fruit_price, i_friut.fruit_color]
    
    mydata_df = pd.DataFrame(create_n_list_in_advance, columns = csv_columns )
    mydata_df.to_csv(csv_file_name)
    return mydata_df

def create_db_database(db_file_name):
    is_exist = os.path.exists(db_file_name)
    if  is_exist:
        os.remove(db_file_name)
    con = sqlite3.connect(db_file_name)
        
    
    cur = con.cursor()
    
    # Create table
    columns_type_list = list(map(lambda x,y: x+' ' + y, csv_columns, columns_type))
    columns_type_list_string = "("+", ".join(map(str, columns_type_list))+")"

    cur.execute(''' 
                CREATE TABLE stocks
                ''' + columns_type_list_string + \
               '''''')
    
    con.commit()
    con.close()
    return con, cur

def fill_db_data_base_using_csv_data_base_with_same_keys(mydata_df):
    con = sqlite3.connect(db_file_name)
    cur = con.cursor()

    for i_csv_row in range(mydata_df.shape[0]):
        # Take row from csv file
        i_row = mydata_df.iloc[i_csv_row]
        i_row_as_list = i_row.to_list()
        
        i_row_as_list_string = "('"+"','".join(map(str, i_row_as_list))+"')"
        #cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT','100')")

        #Insert a csv row of data base
        cur.execute("INSERT INTO stocks VALUES "  \
                    +i_row_as_list_string + \
                    "")
    con.commit()
    con.close()
    return

def print_db_file_info_bsase_single_key(key):
    con = sqlite3.connect(db_file_name)
    cur = con.cursor()
    for row in cur.execute('SELECT * FROM stocks ORDER BY '+ key):
        print(row)
    con.close()
    return

    
def print_db_file_info_base_one_column_and_equal_value(key, value):
    con = sqlite3.connect(db_file_name)
    cur = con.cursor()
    key_index = csv_columns.index(key)
    key_type = columns_type[key_index]
    
    if key_type == 'integer':
        query = f'SELECT * FROM stocks  WHERE {key} = {value}'
    else:
        query = f'SELECT * FROM stocks  WHERE {key} = \'{value}\''
    
    for row in cur.execute(query):
        print(row)

    con.close()
    return 

def print_db_file_info_base_one_column_and_equal_value2(key, value):
    con = sqlite3.connect(db_file_name)
    cur = con.cursor()
    key_index = csv_columns.index(key)
    key_type = columns_type[key_index]
    
    if key_type == 'integer':
        query = f'SELECT FROM stocks  WHERE {key} = {value}'
    else:
        query = f'SELECT FROM stocks  WHERE {key} = \'{value}\''
    
    for row in cur.execute(query):
        print(row)

    con.close()
    return 

def print_db_file_multiplecolumns_base_one_column_and_equal_value(key, value, columns:list):
    con = sqlite3.connect(db_file_name)
    cur = con.cursor()
    key_index = csv_columns.index(key)
    key_type = columns_type[key_index]
    
    if columns is None or len(columns) == 0:
        columns = '*'
    else:
        columns = ','.join(columns)
    
    if key_type == 'integer':
        query = f'SELECT {columns} FROM stocks  WHERE {key} = {value}'
    else:
        query = f'SELECT {columns} FROM stocks  WHERE {key} = \'{value}\''
    
    for row in cur.execute(query):
        print(row)

    con.close()
    return 

def print_db_file_info_base_one_column_and_geater_than_value(key, value):
    con = sqlite3.connect(db_file_name)
    cur = con.cursor()
    key_index = csv_columns.index(key)
    key_type = columns_type[key_index]
    for row in cur.execute('SELECT * FROM stocks  WHERE '+key+'>'+"%s"  % value):
        print(row)
    con.close()
    return 

def print_db_file_info_base_one_column_and_lower_than_value(key, value):
    con = sqlite3.connect(db_file_name)
    cur = con.cursor()
    key_index = csv_columns.index(key)
    key_type = columns_type[key_index]
    for row in cur.execute('SELECT * FROM stocks  WHERE '+key+'<'+"%s"  % value):
        print(row)
    con.close()
    return 

## Task1.1: 
Write Python code to create SQLite database “mydb.db” and create a table “mydata” with the schema of the “mydata.csv”

In [5]:
# init_data_set_configuration settings
init_data_set_configuration()

# create csv file as data frame 
mydata_df = create_csvdatabase_file(max_rows)

# create a db data base
con, cur = create_db_database(db_file_name)

# copy csv data into db file
fill_db_data_base_using_csv_data_base_with_same_keys(mydata_df)

## Task1.2: 
Write Python code to load “mydata.csv” into “mydata” table.

In [6]:
# read csv file 
with open(csv_file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    csv_rows = []
    for row in csv_reader:
        if line_count == 0:
            columns_list = row[1::]
        else:
            current_row = row[1::]
            csv_rows.append(current_row)
        line_count+=1

# create new data frame base csv file 
mydata_df_with_out_pandas = pd.DataFrame(csv_rows, columns = columns_list )
print('The amount of row in the following csv is ' + str(line_count))

The amount of row in the following csv is 11


## Task1.3: 
Write 2 different SQL statements with different condiJons to retrieve different rows. Explain which parts of the statement are predicate and which parts are projecJon.

In [9]:
# print db base single key
print('print_db_file_info_bsase_single_key')
print_db_file_info_bsase_single_key('id')

# 3 option 2 query our data base predicate and projection
print('print_db_file_info_base_one_column_and_equal_value')
print_db_file_info_base_one_column_and_equal_value('color', 'Blue')
print('print_db_file_info_base_one_column_and_geater_than_value')
print_db_file_info_base_one_column_and_geater_than_value('id',2)
print('print_db_file_info_base_one_column_and_lower_than_value')
print_db_file_info_base_one_column_and_lower_than_value('id',2)

print_db_file_info_bsase_single_key
(0, 'Apple', 74, 'Red')
(1, 'Avocado', 82, 'Green')
(2, 'Pineapple', 43, 'Blue')
(3, 'Grape', 99, 'Yellow')
(4, 'Pineapple', 14, 'Yellow')
(5, 'Grape', 76, 'Green')
(6, 'Grape', 57, 'Yellow')
(7, 'Orange', 32, 'Green')
(8, 'Apple', 61, 'Green')
(9, 'Apple', 48, 'Red')
print_db_file_info_base_one_column_and_equal_value
(2, 'Pineapple', 43, 'Blue')
print_db_file_info_base_one_column_and_geater_than_value
(3, 'Grape', 99, 'Yellow')
(4, 'Pineapple', 14, 'Yellow')
(5, 'Grape', 76, 'Green')
(6, 'Grape', 57, 'Yellow')
(7, 'Orange', 32, 'Green')
(8, 'Apple', 61, 'Green')
(9, 'Apple', 48, 'Red')
print_db_file_info_base_one_column_and_lower_than_value
(0, 'Apple', 74, 'Red')
(1, 'Avocado', 82, 'Green')


## 1) Predicate Operation : 

       a)   This operation is used to select rows from a
            table (relation) that specifies a given logic, which is called as
            a predicate. 
       b)  The predicate is a user defined condition to select
           rows of user's choice.

> we used the 4 functions above to filter out rows in which different condition were applied

In [ ]:
print_db_file_multiplecolumns_base_one_column_and_equal_value('color', 'Blue', ['color', 'id'])

## 2) Project Operation : 

        a)  If the user is interested in selecting the values
            of a few attributes, rather than selection all attributes of the Table
            then one should go for PROJECT Operation.  
        
        
> Projection: we used the function above to prject only 2 columns (and also filter some rows)

## Task 2: CSV and Parquet
1. Write Python program that reads “mydata.csv” file and count number of lines
2. By using PyArrow, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapyarrow.parquet”
3. By using Dask, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatadask.parquet”
4. By using Pandas, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapandas.parquet”
5. Examine generated Parquet files. Why do you think Dask generated Parquet file differently than PyArrow and Pandas? What might be explanaJon for this?

## Task 2.1:
Write Python program that reads “mydata.csv” file and count number of lines

## Task 2.2:
By using PyArrow, create Parquet file from the “mydata.csv”. Name Parquet file as

## Task 2.3
By using Dask, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatadask.parquet”

## Task 2.4
By using Pandas, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapandas.parquet”

## Task 2.5
Examine generated Parquet files. Why do you think Dask generated Parquet file differently than PyArrow and Pandas? What might be explanaJon for this?

## Task 3: Split CSV files
1. Write Python code that calculates size of “mydata.csv” in bytes. Define an integer variable “middle” which is the size of “mydata.csv” divided by 2.
2. Write a Python funcJon first_chunk that count number of rows by reading the byte range of the CSV file, from 0 Jll the “middle”. Write a funcJon last_chunk that count number of rows by reading byte range of CSV file from the “middle”+1 Jll the end of the file.
3. Explain why total number of lines from the first chunk and second chunk is larger than the number of lines calculated in the step (1) of Task 2.
4. Suggest an algorithm to resolve the issue from the step (3) and implement it.
5. Check the algorithm of step (4) with mulJple chunks. Define a chunk size to be 16MB. Write a funcJon that process “mydata.csv “ in chunks and count number of lines for each chunk. For example, first chunk will be 0-16MB, second chunk 16MB-32BM, and so on, unJl the last chunk, which might be smaller.

## Task 3.1
Write Python code that calculates size of “mydata.csv” in bytes. Define an integer variable “middle” which is the size of “mydata.csv” divided by 2.

## Task 3.2
Write a Python funcJon first_chunk that count number of rows by reading the byte range of the CSV file, from 0 Jll the “middle”. Write a funcJon last_chunk that count number of rows by reading byte range of CSV file from the “middle”+1 Jll the end of the file.

## Task 3.3
Explain why total number of lines from the first chunk and second chunk is larger than the number of lines calculated in the step (1) of Task 2.

## Task 3.4
Suggest an algorithm to resolve the issue from the step (3) and implement it.

## Task 3.5
Check the algorithm of step (4) with mulJple chunks. Define a chunk size to be 16MB. Write a funcJon that process “mydata.csv “ in chunks and count number of lines for each chunk. For example, first chunk will be 0-16MB, second chunk 16MB-32BM, and so on, unJl the last chunk, which might be smaller.